In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import os
import time

In [2]:
# OTC Markets Website URL
url = "https://www.otcmarkets.com/market-activity/current-market/ALL/active/dollarVolume"

## Scrape data by selenium library

In [3]:
driver = webdriver.Chrome('/usr/local/bin/chromedriver')
driver.get(url)
soup = BeautifulSoup(driver.page_source,'lxml') #Currently, soup only has 25 rows data in web table

In [4]:
# Scrape how much rows in web table
title = soup.title.text
most_active_stocks_no = soup.find("div", {"class": "_12doApv4EC"}).text[17:22]
print(most_active_stocks_no)

18513


In [5]:
#We decide to only collect top 4000 Volume of most active companies.
#We let code to click "MORE" buttomn several times to get top 4000 rows.
#It takes around 6 mins to fully excute this cell code.
for x in range(0, (int(4000/25)-1)):
     
    python_button = driver.find_element_by_class_name('_2sFaw3zGf1')
    python_button.click()
    
    time.sleep(2)


In [6]:
#Need to run soup agian to get top 4000 row data from the browser which is clicked for several times by code.
soup = BeautifulSoup(driver.page_source,'lxml')

In [7]:
OTC_list = []
TIER_list = []

#To scrape TIER link
tier_column = soup.findAll("div", {"class": "_2_IqcDGekT"})
#To scrape Table
table_rows = soup.find_all('tr')

#To scrape the TIER (ps,qb,qx....) from the link
for t in tier_column:
        TIER_list.append(t.img["src"].split("/")[3].split(".")[0])

#To scrape the data in the table
for tr in table_rows:
            td = tr.find_all('td')
            row = [tr.text for tr in td]
            OTC_list.append(row)

In [8]:
# Convert the TIER list into a DataFrame
TIER = pd.DataFrame(TIER_list, columns=["TIER"])
TIER.index += 1
TIER.head()

,TIER
1,ps
2,ps
3,ps
4,ps
5,qb


In [9]:
# Convert the OTC List into Dataframe
OTC = pd.DataFrame(OTC_list, columns=["SYMBOL", "PRICE", "% CHANGE", "$ VOL", "SHARE_VOL", "TRADES"])
OTC = OTC.iloc[1:]
OTC.head()

,SYMBOL,PRICE,% CHANGE,$ VOL,SHARE_VOL,TRADES
1,TCEHY,49.22,+0.24,"141,157,704","2,868,039","5,633"
2,TSCDY,9.85,-0.38,"107,557,436","10,921,536",376
3,BUDFF,87.56,-1.12,"46,022,746","525,455",14
4,NPSNY,50.70,-0.53,"42,512,168","835,607",967
5,CHKVP,505.00,+1.00,"40,305,500","80,010",5


In [10]:
#Merge two data sets into one data set
OTC_Markets = pd.merge(TIER, OTC, left_index=True, right_index=True)
OTC_Markets.head()

,TIER,SYMBOL,PRICE,% CHANGE,$ VOL,SHARE_VOL,TRADES
1,ps,TCEHY,49.22,+0.24,"141,157,704","2,868,039","5,633"
2,ps,TSCDY,9.85,-0.38,"107,557,436","10,921,536",376
3,ps,BUDFF,87.56,-1.12,"46,022,746","525,455",14
4,ps,NPSNY,50.70,-0.53,"42,512,168","835,607",967
5,qb,CHKVP,505.00,+1.00,"40,305,500","80,010",5


In [11]:
# Only keep qx and qb TIER which we want to know
OTC_qxqb_Markets = OTC_Markets[(OTC_Markets.TIER == 'qb') | (OTC_Markets.TIER == 'qx')]
OTC_qxqb_Markets.head()

,TIER,SYMBOL,PRICE,% CHANGE,$ VOL,SHARE_VOL,TRADES
5,qb,CHKVP,505.00,+1.00,"40,305,500","80,010",5
9,qx,RHHBY,32.91,-0.42,"19,289,244","586,755","1,726"
10,qx,CURLF,11.1138,+0.12,"18,999,273","1,714,906","5,390"
11,qx,GBTC,6.85,+1.48,"18,890,855","2,728,799","3,739"
17,qx,OGRMF,7.15,-6.66,"11,905,953","1,628,362","4,347"


In [12]:
# Reset Index
OTC_qxqb_Markets.reset_index(drop=True,inplace=True)
OTC_qxqb_Markets.index += 1
OTC_qxqb_Markets.head()

,TIER,SYMBOL,PRICE,% CHANGE,$ VOL,SHARE_VOL,TRADES
1,qb,CHKVP,505.00,+1.00,"40,305,500","80,010",5
2,qx,RHHBY,32.91,-0.42,"19,289,244","586,755","1,726"
3,qx,CURLF,11.1138,+0.12,"18,999,273","1,714,906","5,390"
4,qx,GBTC,6.85,+1.48,"18,890,855","2,728,799","3,739"
5,qx,OGRMF,7.15,-6.66,"11,905,953","1,628,362","4,347"


In [13]:
#export csv to be backup
OTC_qxqb_Markets.to_csv("OTCmarkets4000_qx_qb.csv")

## Convert dataframe into Mongodb

In [ ]:
import pandas as pd
from pymongo import MongoClient
import json
import os
import datetime

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.otc_markets
collection = db.most_active

In [ ]:
records = json.loads(OTC_Markets.T.to_json()).values()
db.most_active.insert_many(OTC_Markets.to_dict('records'))